### This Python notebook should sreve as an example of how to use tint
First import all modules that are needed

In [1]:
from tint import Cell_tracks, animate
import os, pandas as pd
from itertools import groupby
import numpy as np
from netCDF4 import Dataset as nc, num2date, date2num
from datetime import datetime, timedelta
from tint.helpers import get_times, get_grids
from tint.visualization import embed_mp4_as_gif

### Define some important variables

In [2]:
dataF = os.path.join(os.path.abspath('.'),'data','CPOL_radar.nc') #NetCDF data file
trackdir = os.path.join(os.path.abspath('.'),'tracks') #Output directory
overwrite = True #Overwirte existing old files
first = '2006-11-16 03:00' #Start-date
last = '2006-11-16 11:00' #End-date

### Open the netCDF file and apply the tracking

In [ ]:
with nc(dataF) as ncf:
        slices = get_times(ncf.variables['time'], first, last,
                           ncf.variables['isfile'][:]) #Get the data
        lats = ncf.variables['latitude'][:,0]
        lons = ncf.variables['longitude'][0,:]
        x = lons[int(117/2)]
        y = lats[int(117/2)]
        grids = []
        for s in slices:
            ani = False
            #Create an iterator for the data dictionary 
            gr = (i for i in get_grids(ncf, s, lons, lats, varname='radar_estimated_rain_rate'))
            anim = (i for i in get_grids(ncf, s, lons, lats, varname='radar_estimated_rain_rate'))
            #Construct start and end date of slcie
            start = num2date(ncf.variables['time'][s[0]],
                             ncf.variables['time'].units)
            end = num2date(ncf.variables['time'][s[-1]],
                           ncf.variables['time'].units)
            #Filename suffix
            suffix = '%s-%s'%(start.strftime('%Y_%m_%d_%H'), end.strftime('%Y_%m_%d_%H'))
            tracks_obj = Cell_tracks()
            tracks_obj.params['MIN_SIZE'] = 4
            tracks_obj.params['FIELD_THRESH'] = 1
            track_file = os.path.join(trackdir,'tint_tracks_%s.pkl'%suffix)
            if not os.path.isfile(track_file) or overwrite:
                ncells = tracks_obj.get_tracks(gr, (x,y))
                if ncells > 2 :
                    tracks_obj.tracks.to_pickle(track_file)
                    ani = True
                else:
                    ani = False
            animate(tracks_obj, anim, os.path.join(trackdir,'ani', 'tint_tracks_%s.mp4'%suffix), overwrite=overwrite, dt = 9.5)
            
               
                
            

Writing tracks for scan 0
Writing tracks for scan 1
Writing tracks for scan 2
Writing tracks for scan 3
Writing tracks for scan 4
Writing tracks for scan 5
Writing tracks for scan 6
Writing tracks for scan 7
Writing tracks for scan 8
Writing tracks for scan 9
Writing tracks for scan 10
Writing tracks for scan 11
Writing tracks for scan 12
Writing tracks for scan 13
Writing tracks for scan 14
Writing tracks for scan 15
Writing tracks for scan 16
Writing tracks for scan 17
Writing tracks for scan 18
Writing tracks for scan 19
Writing tracks for scan 20
Writing tracks for scan 21
Writing tracks for scan 22
Writing tracks for scan 23
Writing tracks for scan 24
Writing tracks for scan 25
Writing tracks for scan 26
Writing tracks for scan 27
Writing tracks for scan 28
Writing tracks for scan 29
Writing tracks for scan 30
Writing tracks for scan 31
Writing tracks for scan 32
Writing tracks for scan 33
Writing tracks for scan 34
Writing tracks for scan 35
Writing tracks for scan 36
Writing tra

/usr/lib/python3/dist-packages/matplotlib/colors.py:581: RuntimeWarning: invalid value encountered in less
  cbook._putmask(xa, xa < 0.0, -1)


Frame: 1
Frame: 2
Frame: 3
Frame: 4
Frame: 5
Frame: 6
Frame: 7
Frame: 8
Frame: 9
Frame: 10
Frame: 11
Frame: 12
Frame: 13
Frame: 14
Frame: 15
Frame: 16
Frame: 17
Frame: 18
Frame: 19
Frame: 20
Frame: 21
Frame: 22
Frame: 23
Frame: 24


In [ ]:
embed_mp4_as_gif(os.path.join(trackdir,'ani', 'tint_tracks_%s.mp4'%suffix))